# JSON model definition and folder import
Often, it is more convenient to define all (or many) of the model parameters in input files rather than in scripts. To simplify this process, a support function `wawi.io.import_folder` is provided in WAWI.

This imports all files in the specified folder. The file names of the relevant files in the provided path are defined as inputs to the function (default values in parentheses): 

- pontoon_stl ('pontoon.stl')
- aero_sections ('aero_sections.json')
- modal ('modal.json')
-  pontoon('pontoon.json')
- eldef ('element.json')
- orientations ('orientations.json')
- drag_elements ('drag_elements.json')
- pontoon_types ('pontoon_tbe defined.'):

## Modal definition (modal)

## Pontoon and pontoon types (pontoon, pontoon_types, pontoon_stl)

## Element definition (eldef, orientations)

## Aerodynamic sections (aero_sections)

## Drag elements (drag_elements)

## Example import
The folder ./model/ contains the following files:

![](files.png "Files")

Note that these are named as the default values of all the relevant inputs, so they are therefore not defined below. The following code snippet imports the folder and returns a WAWI `Model`-object.

In [13]:
import wawi
from wawi.io import import_folder

model = import_folder('./model/', sort_nodes_by_x=True)

This model contains a curved floating bridge with columns. No sections in the `element.json` file matches the section definitions in the `drag_elements.json` file, so no drag elements are used (the file is still provided to indicate how it can be defined).

In [3]:
model.plot(tmat_on=['undeformed'])

Widget(value='<iframe src="http://localhost:56527/index.html?ui=P_0x1950f07c2f0_0&reconnect=auto" class="pyvis…

In [19]:
# Plot dry mode with index 1
model.plot_mode(1, use_dry=True)

Widget(value='<iframe src="http://localhost:56527/index.html?ui=P_0x19523d5ca70_2&reconnect=auto" class="pyvis…